In [43]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\heavy\Documents\PythonScripts\Challenge Globant\praxis-road-356512-f4b3c65868bc.json"

In [44]:
class GlobalVars:
    dataset = 'challengeglobant'
    project = 'praxis-road-356512'
    bucket_name = "challengeglobant"
    file_path = "raw/{file_name}.csv"
    file_config = {"departments":["id","department"], "hired_employees": ["id","name","datetime","department_id", "job_id"]
                   , "jobs": ["id", "job"]}
    

In [60]:
from google.cloud import storage, bigquery
# import datalab.bigquery as bq
# import pandas_gbq as gbq
import pandas as pd
from datetime import datetime, timedelta, date
import io

class ManageData:

    @staticmethod
    def to_bq(df, table_id, project, today_file_date=None, schema=None):
        """
        df = dataframe to upload
        table_id = dataset + table name
        project = project to upload data
        today_file_date = optional var to add to dataframe that upload
        schema = optional var to define column types and names
        """

        try:
            today_date = datetime.today()  # '2022-01-06'
            today_file_date = str(today_date.strftime('%Y-%m-%d'))
        except Exception as e:
            print(e)
            today_date = datetime.today()  # -timedelta(hours=5)
            today_file_date = str(today_date.strftime('%Y-%m-%d'))

        try:
            # get client
            print("Carga Datos BQ")
            bq_client = bigquery.Client(project=project)
            
            # add patition of date
            #df["upload_date"] = pd.to_datetime(today_date)

            if schema is None:
                # save to BQ
                job = bq_client.load_table_from_dataframe(df, table_id)
                # Wait for the job to complete.
                job.result()
            else:
                print("else upload bq")
                job = bq_client.load_table_from_dataframe(df, table_id, job_config=schema)
                job.result()

        except Exception as e:
            print(f"error loading information to bq. please check the process manually: {e}")
        return

    @staticmethod
    def read_csv_from_gcs(bucket_name, file_path, col_file_name):
        storage_client = storage.Client()

        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(file_path)

        csv_data = blob.download_as_text()

        dataframe = pd.read_csv(io.StringIO(csv_data), names=GlobalVars.file_config.get(col_file_name))

        return dataframe


    @staticmethod
    def upload_storage(bucket_gcs, df, path_filename):
        try:
            client = storage.Client()
            bucket = storage_client.get_bucket(bucket_gcs)
            bucket.blob(path_filename, chunk_size=262144).upload_from_string(df.to_csv(), 'text/csv')
        except Exception as e:
            print(f"Error loading Dataframe to Storage, please check: {e}")
        return


In [61]:

# Read the CSV file and load it into a DataFrame
for key, value in GlobalVars.file_config.items():
    col_file_name = key
    print(col_file_name)
    file_path = GlobalVars.file_path.format(file_name=col_file_name)
    df = ManageData.read_csv_from_gcs(GlobalVars.bucket_name, file_path, col_file_name)
    df = df.reset_index(drop=True)
    table = file_path.split('.')[0]
    print(table)
    table_id = f"{GlobalVars.dataset}.{table}"
    
    if col_file_name == 'hired_employees':
        print(df.count())
        df = df.dropna()
        print(df.count())
        df['department_id'] = df['department_id'].astype(int)
        df['job_id'] = df['job_id'].astype(int)
        ManageData.to_bq(df, table_id, GlobalVars.project)
    else:
        ManageData.to_bq(df, table_id, GlobalVars.project)


departments
departments
Carga Datos BQ
hired_employees
hired_employees
id               1999
name             1980
datetime         1985
department_id    1978
job_id           1983
dtype: int64
id               1929
name             1929
datetime         1929
department_id    1929
job_id           1929
dtype: int64
Carga Datos BQ
jobs
jobs
Carga Datos BQ
